# ML Model for Actionable item classification

## 1. Load data

In [2]:
import pandas as pd

In [8]:
#Load data
actions_df = pd.read_csv('actions.csv', names = ['action_sent'])

### 1.1 explore the dataset

In [10]:
pd.options.display.max_colwidth = 1500
actions_df

,action_sent
0,Activate all who work with Transmission or have any good ideas on the subject.
1,Add more to your score by stopping in and picking up hefty load of construction supplies to win.
2,Add O'neal Winfee and George Smith to the attendees list.
3,"Additionally, send me the payment schedule for Tenaska IV this month."
4,Adjust our purchase amount from each party based on the transport allocation.
...,...
1245,Write me note about what is going on and what issues you need my help to deal with when you send the rentroll.
1246,Write verification plans specifications and documentation today and send me.
1247,you have to expand on the maintenance tools.
1248,You have to resolve Enron's ongoing concerns at any cost.


- The tagged data available is only of one class i.e action class
- 